In [ ]:
import json
import multiprocessing
from pathlib import Path
import sys
import time

import numpy as np
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
import matplotlib.pyplot as plt
from IPython import display

sys.path.append(str(Path.cwd()/"run"))

import common as cm
import memory_lookup as ml
from tetris_gym import TetrisGymEnv, make_env

In [ ]:
%matplotlib inline

In [ ]:
def show_state(dummyenv: DummyVecEnv, step=0, info=""):
    env = dummyenv.envs[0]
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(), cmap='gray', vmin=0, vmax=255)
    plt.title("%s | Step: %d %s" % ("Tetris", step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

In [ ]:
Path.cwd()

In [ ]:
# Select the latest model
# model_paths = [fp for fp in cm.TRAINING_SESSION_DIR.glob("*.zip")]
# model_paths.sort(reverse=True)
# model_path = model_paths[0]

model_path = Path.cwd() / 'data/logs/best_model.zip'

print(f"Loading model {model_path}")

config_path = cm.CONFIG_DIR / "env_default.json"
with open(config_path, "rt") as F:
    env_config = json.load(F)
    F.close()

# DummyVecEnv lets us run the multi cpu env w/ 1 env
env = DummyVecEnv([make_env(0, env_config)])
model = A2C.load(model_path, env=env)

In [ ]:
obs = env.reset()
tot_reward = 0
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    tot_reward += reward
    show_state(env, i, tot_reward)
    # print(np.reshape(obs["tilemap"][0, :], (-1, 10)))
    # print(obs["rgb_array"][0][:, :, 0])
    # print("height", obs["height"][0, 0])
    # print("next_shape", ml.SHAPE_LOOKUP.get(obs["next_shape"][0], None))
    # print(info)
    # print(i)
    # time.sleep(0.001)